In [1]:
import numpy as np
import matplotlib.pyplot as plt
from Cumulant4_fonction import C4_long, C4_court, Cross_time
from DoubleRigidWallOverdampedLangevin3D_cython import RigidWallOverdampedLangevin3D
from scipy.integrate import quad
from scipy.io import loadmat
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from numpy import trapz


def cm2inch(value):
    return value / 2.54

from tqdm import tqdm
# axes.xaxis.set_tick_params(direction='in', which='both')
# axes.yaxis.set_tick_params(direction='in', which='both')


mpl.rcParams["xtick.direction"] = "in"
mpl.rcParams["ytick.direction"] = "in"
mpl.rcParams["lines.markeredgecolor"] = "k"
mpl.rcParams["lines.markeredgewidth"] = 1
mpl.rcParams["figure.dpi"] = 200
from matplotlib import rc

rc("font", family="serif")
rc("text", usetex=False)
rc("xtick", labelsize="medium")
rc("ytick", labelsize="medium")
rc("axes", labelsize="large")

# Étude de la convergence du C4 (fake diff) en fonction de $\mathrm{d}t$, $l_\mathrm{B}$ et $Nt_\mathrm{sub}$

In [3]:
#Paramètres et fonctions:

H = 1.5e-6
a = 0.5e-6
Hs = H+0.1e-6 # Hs > H
Hp = H-a
kBT = 4e-21
beta = 1/kBT

# lB = [0.1e-6, 0.2e-6, 0.5e-6]
# Nt_sub = [1, 10, 100]
# dt = [1e-5, 1e-3]

Nt = int(10000000)

Db = kBT/(6*np.pi*0.001*a)
D_para0 = (H-a)/(2*a) * Db
D_perp0 = (H-a)/(2*a) * Db


def Dpara(z):
    """
    D parallèle en fonction de z.
    Return: une liste de Dpara de meme taille que z.
    """
    return (D_para0*(1-(z/Hs)**2))

def Dperp(z):
    """
    D perpendiculaire en fonction de z.
    Return: une liste de Dperp de meme taille que z.
    D=0 lorsque z = +/-(H-a)
    """
    return D_perp0*(1-(z/(H-a))**2)

def V(z):
    """
    Potentiel subit par la particule en fonction de z.
    Return: une liste de V de meme taille que z.
    """
    return 0#z/(beta*lB)

def _Peq(z):
    P_eq = np.exp(-beta*V(z))
    N = quad(lambda zz: np.exp(-beta*V(zz)), -Hp, Hp)[0]
    
    if z > Hp:
        return 0
    elif z < -Hp:
        return 0
    else:
        return P_eq / N

def Peq(z):
    
    if type(z) == float:
        return _Peq(z)
    z = np.array(z)
    return np.array([_Peq(zz) for zz in z])
    
def _Pdz(dz):
    
    return 1 / (4 * Hp**2) * (2 * Hp - np.abs(dz))

def P_dz(dz):
    
    return np.array([_Pdz(i) for i in dz])

zz = np.linspace(-Hp, Hp, 1000)
dz = np.linspace(-2*Hp, 2*Hp, 1000)

Pdz = P_dz(dz)   
    

In [4]:
#Théorie asymptotique

D4, C4 = C4_long(Dpara=Dpara, Dperp=Dperp, V=V, kBT=kBT, a=-H+a, b=H-a)
A4 = C4_court(Dpara=Dpara, V=V, kBT=kBT, a=-H+a, b=H-a)

#Theorie pour tout temps

C4_tot_theo = lambda t: (2 * t * np.power(H-a,6) * np.power(D_para0,2)) / (135 * D_perp0 * np.power(Hs,4)) - (
    np.power(H-a,8) * np.power(D_para0,2)
) / (405 * np.power(D_perp0,2) * np.power(Hs,4)) * (1 - np.exp(-(6 * D_perp0 * t) / (np.power(H-a,2))))


In [5]:
import csv
